In [12]:
!pip3 install camelot-py[cv] tabula-py

In [13]:
!pip install pikepdf

In [14]:
!pip install xlwt

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import xlwt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import camelot 
import tabula
import pikepdf
#pdf = pikepdf.open(file,password='ABDPM5030G')
#pdf.save('book_without_pass.pdf')
import warnings
warnings.filterwarnings('ignore')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
/kaggle/input/contract-notes/2022-03-08-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-09-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-07-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-17-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-16-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-15-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-11-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-10-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-14-contract-notes_DM3955.pdf


### Starting the PDF table Extraction, and the helper function used for the same

In [16]:
def clean_df(my_table):
    my_table = my_table[2:]
    my_table.drop([0,1,3,8,12],axis=1,inplace=True)
    return my_table

In [17]:
def select_tables(ca_table):
    calc_tab = pd.DataFrame()
    for table in ca_table:
        if table.df.shape[1] == 13:
            calc_tab = pd.concat([calc_tab,table.df])
    calc_tab = clean_df(calc_tab)
    return calc_tab

In [18]:
def correct_txn(dataframe):
    for index,trans in enumerate(dataframe.transaction):
        if '\n' in trans:
            txn_len = len(trans.split('\n'))
            #print(index)
            for i in range(txn_len):
                dataframe.loc[index+i,'transaction'] = trans.split('\n')[i]
    return dataframe

In [19]:
def correct_qty(dataframe):
    for index,qty in enumerate(dataframe.qty):
        if '\n' in qty:
            txn_len = len(qty.split('\n'))
            #print(index)
            for i in range(txn_len):
                dataframe.loc[index+i,'qty'] = qty.split('\n')[i]
    return dataframe

In [20]:
#The below function takes the raw file and outputs the cleaned dataframe
def get_data(file):
    pdf = pikepdf.open(file,password='ABDPM5030G')
    pdf.save('book_without_pass.pdf')
    ca_tables = camelot.read_pdf('./book_without_pass.pdf',pages='all')
    my_data = select_tables(ca_tables)
    my_data.columns = ['order_no','script','transaction',
                   'qty','price','price_d','price_close','PnL']
    my_data = my_data[my_data.order_no != '']
    my_data = my_data.reset_index()
    my_data.drop('index',axis=1,inplace=True)
    my_data = correct_qty(my_data)
    my_data = correct_txn(my_data)
    my_data.qty = my_data.qty.astype('int')
    my_data.price = my_data.price.astype('float')
    return my_data # Try to take the raw data as much as possible, to avoid losing control

In [21]:
def todays_forward(carry_forward, day_df):
    trade_count = day_df.groupby(['script','transaction'])['qty'].sum().reset_index()
    #Cancelling out the trades of the day
    traded_scripts = trade_count.script.unique()
    transaction = []
    txn_qty = []
    for script in traded_scripts:
        temp = trade_count[trade_count.script == script]
        #which is more?
        if (temp.qty.values[0] > temp.qty.values[-1]):
            x = temp.qty.values[0] - temp.qty.values[-1]
            transaction.append('B')
            txn_qty.append(x)
        else:    
            x = temp.qty.values[-1] - temp.qty.values[0]
            transaction.append('S')
            txn_qty.append(x)
    #Creating the pending trades of the day
    traded_df = pd.DataFrame({'script' : traded_scripts,
                         'txn_type' : transaction,
                         'txn_qty' : txn_qty
                         })
    final_df = traded_df.merge(right=carry_f,left_on='script',right_on='script',how='left')
    final_df['remaining_qty'] = abs(final_df.txn_qty - final_df.qty)
    final_df.drop(['order_no','price','price_d','price_close','PnL'],axis=1,inplace=True)
    return final_df

In [22]:
#decide the script
def locate_script(day_df,script,qty,transaction):
    locating = day_df[(day_df.script == script)&(day_df.transaction == transaction)]
    return locating.iloc[-qty:,:]
            #day_dy[-3:,:] will only work

### Now we know which scripts are finally pending at the day's end

Locate the final trades in that script

In [23]:
#Next is to introduce the scripts that are carried forward for the day. 
#This function to makes the dataframe for the carried forward to this day.

def carried_forward(order_no,script,transaction,qty,price):
    loc_qty = 1
    values=pd.DataFrame({'order_no':order_no,'script':script,
                         'transaction':transaction,'qty':loc_qty,
                         'price':price},index=[0])
    values = values.reindex(list(range(qty)),method='ffill')
    return values

In [24]:
def insert_row(idx, df, df_insert):
    dfA = df.iloc[:idx, ]
    dfB = df.iloc[idx:, ]

    df = dfA.append(df_insert).append(dfB).reset_index(drop = True)

    return df

In [25]:
def make_single_1(dataf,qty,index):
    values = dataf.loc[index]
    values[3] = 1 #Here i am changing the qty to 1
    values=pd.DataFrame([values.values],columns=values.index)
    values = values.reindex(list(range(index,index+qty)),method='ffill')
    return values

In [66]:
#This function finally provides the corrected dataframe for each script, transaction wise
def get_trades(df,script):
    df = df[df.order_no != '0'] #Don't take the forwarded trades
    temp = df[df.script == script]
    buy = temp[temp.transaction == 'B'].reset_index()
    buy.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
    buy.order_no = buy.order_no.astype('int') 
    sell = temp[temp.transaction == 'S'].reset_index()
    sell.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
    sell.order_no = sell.order_no.astype('int') 
    mult_buy = buy[buy.qty > 1]
    mult_sell = sell[sell.qty > 1]
    buy.drop(mult_buy.index,inplace=True)
    sell.drop(mult_sell.index,inplace=True)
    
    #calculate sell items
    for i,qty in mult_sell['qty'].iteritems():
        temp = make_single_1(dataf=mult_sell,qty=qty,index=i)
        update_buy = insert_row(i,sell,temp)
        sell = update_buy
        #print('exit loop')

    #calculate buy items
    for i,qty in mult_buy['qty'].iteritems():
        temp = make_single_1(dataf=mult_buy,qty=qty,index=i)
        update_buy = insert_row(i,buy,temp)
        buy = update_buy
        #print('exit loop')
    
    buy.sort_values('order_no',inplace=True)
    sell.sort_values('order_no',inplace=True)
    
    return buy,sell

In [27]:
contact17 = get_data('../input/contract-notes/2022-03-17-contract-notes_DM3955.pdf')
contact17.script.unique()

2022-03-18T15:21:41 - INFO - Processing page-1
2022-03-18T15:21:45 - INFO - Processing page-2
2022-03-18T15:21:48 - INFO - Processing page-3


In [40]:
buy_apr,sell_apr = get_trades(df=contact17,script='SILVERMIC22APRFUT')

In [49]:
apr_forward = carried_forward(order_no='forward',script='SILVERMIC22APRFUT',transaction='S',
                              qty=6,price=67517)
sell_apr = insert_row(0,sell_apr,apr_forward)

In [50]:
apr_forward_close = carried_forward(order_no='close',script='SILVERMIC22APRFUT',transaction='B',
                              qty=6,price=68625)
buy_apr = insert_row(0,buy_apr,apr_forward_close)

In [51]:
final_comp_APR = buy_apr.merge(right=sell_apr,right_index=True,left_index=True,how='left')
final_comp_APR['PnL'] = final_comp_APR.price_y - final_comp_APR.price_x
final_comp_APR.to_excel('Apr_Script.xls')

In [45]:
buy_jun,sell_jun = get_trades(df=contact17,script='SILVERMIC22JUNFUT')

In [46]:
jun_forward = carried_forward(order_no='forwarded',script='SILVERMIC22JUNFUT',transaction='B',
                              qty=6,price=68327)
jun_forward_close= carried_forward(order_no='close',script='SILVERMIC22JUNFUT',transaction='B',
                              qty=6,price=69410)

In [53]:
sell_jun = insert_row(0,sell_jun,jun_forward)
buy_jun = insert_row(0,buy_jun,jun_forward_close)

final_comp_Jun = buy_jun.merge(right=sell_jun,right_index=True,left_index=True,how='left')
final_comp_Jun['PnL'] = final_comp_Jun.price_x - final_comp_Jun.price_y
final_comp_Jun.to_excel('Jun_Script.xls')

#### Starting to test with other contacts...

In [87]:
contact07 = get_data('../input/contract-notes/2022-03-07-contract-notes_DM3955.pdf')
contact07.script.unique()

2022-03-18T16:23:46 - INFO - Processing page-1
2022-03-18T16:23:50 - INFO - Processing page-2
2022-03-18T16:23:53 - INFO - Processing page-3
2022-03-18T16:23:55 - INFO - Processing page-4


array(['SILVERM22APRFUT', 'SILVERM22JUNFUT', 'SILVERMIC22APRFUT',
       'SILVERMIC22JUNFUT'], dtype=object)

In [93]:
buy_apr,sell_apr = get_trades(df=contact07,script='SILVERMIC22JUNFUT')
apr_forward = carried_forward(order_no='forward',script='SILVERMIC22JUNFUT',transaction='S',
                              qty=2,price=70758)
sell_apr = insert_row(0,sell_apr,apr_forward)
#apr_forward_close = carried_forward(order_no='close',script='SILVERMIC22APRFUT',transaction='S',qty=2,price=70069)
#buy_apr = insert_row(0,buy_apr,apr_forward)
if (buy_apr.shape[0] != 0) and (sell_apr.shape[0] != 0):
    final_comp_APR = buy_apr.merge(right=sell_apr,right_index=True,left_index=True,how='left')
    final_comp_APR['PnL'] = final_comp_APR.price_y - final_comp_APR.price_x
    print('entered')
elif buy_apr.shape[0] != 0:
    final_comp_APR = buy_apr
    print('entered elif')
else:
    final_comp_APR = sell_apr
    print('entered else')

entered else


In [94]:
final_comp_APR

,order_no,script,transaction,qty,price
0,forward,SILVERMIC22JUNFUT,S,1,70758.0
1,forward,SILVERMIC22JUNFUT,S,1,70758.0
2,210171855,SILVERMIC22JUNFUT,S,1,70387.0
3,210206727,SILVERMIC22JUNFUT,S,1,70787.0


In [105]:
def get_script_frame(dataf,script,fwd_txn,fwd_qty,fwd_price):
    buy_scr,sell_scr = get_trades(df=dataf,script=script)
    print(buy_scr.shape,sell_scr.shape)
    scr_forward = carried_forward(order_no='forward',script=script,transaction=fwd_txn,
                                  qty=fwd_qty,price=fwd_price)
    print(scr_forward.shape)
    #scr_forward_close = carried_forward(order_no='close',script=script,transaction=cls_txn,
                                  #qty=cls_qty,price=cls_price)
    if fwd_txn == 'B':
        buy_scr = insert_row(0,buy_scr,scr_forward)
        #sell_scr = insert_row(0,sell_scr,scr_forward_close)
    else:
        sell_scr = insert_row(0,sell_scr,scr_forward)
        #buy_scr = insert_row(0,buy_scr,scr_forward_close)
    print('final trade script buy and sell',buy_scr.shape,sell_scr.shape)
    if (buy_scr.shape[0] != 0) and (sell_scr.shape[0] != 0):
        if fwd_txn == 'B': #The buy script forward has to be on the top
            final_comp_scr = buy_scr.merge(right=sell_scr,right_index=True,left_index=True,how='left')
            final_comp_scr['PnL'] = final_comp_scr.price_y - final_comp_scr.price_x
        else:#The sell script forward has to be on the top
            final_comp_scr = sell_scr.merge(right=buy_scr,right_index=True,left_index=True,how='left')
            final_comp_scr['PnL'] = final_comp_scr.price_x - final_comp_scr.price_y
        print('entered')
        return final_comp_scr
        
    elif buy_scr.shape[0] != 0:
        final_comp_scr = buy_scr
        print('entered elif')
        return final_comp_scr
    else:
        final_comp_scr = sell_scr
        print('entered else')
        return final_comp_scr
    

In [106]:
apr_07 = get_script_frame(dataf=contact07,script='SILVERMIC22APRFUT',
                          fwd_txn='B',fwd_qty=2,fwd_price=69257)
apr_07.head()

(22, 5) (20, 5)
(2, 5)
final trade script buy and sell (24, 5) (20, 5)
entered


,order_no_x,script_x,transaction_x,qty_x,price_x,order_no_y,script_y,transaction_y,qty_y,price_y,PnL
0,forward,SILVERMIC22APRFUT,B,1,69257.0,210005892.0,SILVERMIC22APRFUT,S,1.0,70440.0,1183.0
1,forward,SILVERMIC22APRFUT,B,1,69257.0,210017029.0,SILVERMIC22APRFUT,S,1.0,70625.0,1368.0
2,210002650,SILVERMIC22APRFUT,B,1,70292.0,210059932.0,SILVERMIC22APRFUT,S,1.0,70841.0,549.0
3,210002774,SILVERMIC22APRFUT,B,1,70298.0,210062808.0,SILVERMIC22APRFUT,S,1.0,71002.0,704.0
4,210006089,SILVERMIC22APRFUT,B,1,70445.0,210066794.0,SILVERMIC22APRFUT,S,1.0,71180.0,735.0


In [107]:
jun_07 = get_script_frame(dataf=contact07,script='SILVERMIC22JUNFUT',
                          fwd_txn='S',fwd_qty=2,fwd_price=70758)
jun_07

(0, 5) (2, 5)
(2, 5)
final trade script buy and sell (0, 5) (4, 5)
entered else


,order_no,script,transaction,qty,price
0,forward,SILVERMIC22JUNFUT,S,1,70758.0
1,forward,SILVERMIC22JUNFUT,S,1,70758.0
2,210171855,SILVERMIC22JUNFUT,S,1,70387.0
3,210206727,SILVERMIC22JUNFUT,S,1,70787.0


In [100]:
contact08 = get_data('../input/contract-notes/2022-03-08-contract-notes_DM3955.pdf')
contact08.script.unique()

2022-03-18T16:35:11 - INFO - Processing page-1
2022-03-18T16:35:15 - INFO - Processing page-2
2022-03-18T16:35:21 - INFO - Processing page-3
2022-03-18T16:35:25 - INFO - Processing page-4
2022-03-18T16:35:27 - INFO - Processing page-5


array(['SILVERM22APRFUT', 'SILVERM22JUNFUT', 'SILVERMIC22APRFUT',
       'SILVERMIC22JUNFUT'], dtype=object)

In [108]:
jun_08 = get_script_frame(dataf=contact08,script='SILVERMIC22JUNFUT',
                          fwd_txn='S',fwd_qty=4,fwd_price=72211)
#jun_07
apr_08 = get_script_frame(dataf=contact08,script='SILVERMIC22APRFUT',
                          fwd_txn='B',fwd_qty=4,fwd_price=71418)
#apr_07.head()

(16, 5) (15, 5)
(4, 5)
final trade script buy and sell (16, 5) (19, 5)
entered
(28, 5) (29, 5)
(4, 5)
final trade script buy and sell (32, 5) (29, 5)
entered


In [109]:
jun_08

,order_no_x,script_x,transaction_x,qty_x,price_x,order_no_y,script_y,transaction_y,qty_y,price_y,PnL
0,forward,SILVERMIC22JUNFUT,S,1,72211.0,210001228.0,SILVERMIC22JUNFUT,B,1.0,70715.0,1496.0
1,forward,SILVERMIC22JUNFUT,S,1,72211.0,210028135.0,SILVERMIC22JUNFUT,B,1.0,71395.0,816.0
2,forward,SILVERMIC22JUNFUT,S,1,72211.0,210065801.0,SILVERMIC22JUNFUT,B,1.0,72334.0,-123.0
3,forward,SILVERMIC22JUNFUT,S,1,72211.0,210082837.0,SILVERMIC22JUNFUT,B,1.0,71802.0,409.0
4,210004553,SILVERMIC22JUNFUT,S,1,70917.0,210102352.0,SILVERMIC22JUNFUT,B,1.0,71981.0,-1064.0
5,210015746,SILVERMIC22JUNFUT,S,1,70971.0,210105300.0,SILVERMIC22JUNFUT,B,1.0,71885.0,-914.0
6,210021177,SILVERMIC22JUNFUT,S,1,71349.0,210000762.0,SILVERMIC22JUNFUT,B,1.0,70714.0,635.0
7,210034964,SILVERMIC22JUNFUT,S,1,71768.0,210000763.0,SILVERMIC22JUNFUT,B,1.0,70715.0,1053.0
8,210046250,SILVERMIC22JUNFUT,S,1,72326.0,210011954.0,SILVERMIC22JUNFUT,B,1.0,70799.0,1527.0
9,210075317,SILVERMIC22JUNFUT,S,1,72248.0,210025309.0,SILVERMIC22JUNFUT,B,1.0,71551.0,697.0


In [110]:
apr_08

,order_no_x,script_x,transaction_x,qty_x,price_x,order_no_y,script_y,transaction_y,qty_y,price_y,PnL
0,forward,SILVERMIC22APRFUT,B,1,71418.0,210000790.0,SILVERMIC22APRFUT,S,1.0,70015.0,-1403.0
1,forward,SILVERMIC22APRFUT,B,1,71418.0,210000791.0,SILVERMIC22APRFUT,S,1.0,70013.0,-1405.0
2,forward,SILVERMIC22APRFUT,B,1,71418.0,210001192.0,SILVERMIC22APRFUT,S,1.0,70017.0,-1401.0
3,forward,SILVERMIC22APRFUT,B,1,71418.0,210028358.0,SILVERMIC22APRFUT,S,1.0,70685.0,-733.0
4,210025410,SILVERMIC22APRFUT,B,1,70869.0,210041803.0,SILVERMIC22APRFUT,S,1.0,71300.0,431.0
5,210036899,SILVERMIC22APRFUT,B,1,71134.0,210043218.0,SILVERMIC22APRFUT,S,1.0,71323.0,189.0
6,210049696,SILVERMIC22APRFUT,B,1,71408.0,210065711.0,SILVERMIC22APRFUT,S,1.0,71561.0,153.0
7,210067015,SILVERMIC22APRFUT,B,1,71459.0,210084159.0,SILVERMIC22APRFUT,S,1.0,71355.0,-104.0
8,210075175,SILVERMIC22APRFUT,B,1,71654.0,210088703.0,SILVERMIC22APRFUT,S,1.0,70974.0,-680.0
9,210082297,SILVERMIC22APRFUT,B,1,71103.0,210091973.0,SILVERMIC22APRFUT,S,1.0,71420.0,317.0


In [111]:
import os
directory = '../input/contract-notes'
scripts = ['SILVERMIC22JUNFUT','SILVERMIC22APRFUT']
APR_FWD = [70069,71418,69692,70500,70412,68993,68466,67517,68625]
JUN_FWD = [70758,72211,70443,71246,71170,69805,69256,68327,69410]

In [123]:
dates = []
files = []

In [124]:
for filename in os.listdir(directory):
    files.append(filename)
    temp = filename.split('-')
    new_date = temp[0]+'-'+temp[1]+'-'+temp[2]
    dates.append(new_date)

In [132]:
contract_df = pd.DataFrame({'filename':files,'dates':dates})
contract_df.dates = pd.to_datetime(contract_df.dates)
contract_df.sort_values('dates',inplace=True)
contract_df['Jun_close'] = JUN_FWD
contract_df['Apr_close'] = APR_FWD
contract_df

,filename,dates,Jun_close,Apr_close
2,2022-03-07-contract-notes_DM3955.pdf,2022-03-07,70758,70069
0,2022-03-08-contract-notes_DM3955.pdf,2022-03-08,72211,71418
1,2022-03-09-contract-notes_DM3955.pdf,2022-03-09,70443,69692
7,2022-03-10-contract-notes_DM3955.pdf,2022-03-10,71246,70500
6,2022-03-11-contract-notes_DM3955.pdf,2022-03-11,71170,70412
8,2022-03-14-contract-notes_DM3955.pdf,2022-03-14,69805,68993
5,2022-03-15-contract-notes_DM3955.pdf,2022-03-15,69256,68466
4,2022-03-16-contract-notes_DM3955.pdf,2022-03-16,68327,67517
3,2022-03-17-contract-notes_DM3955.pdf,2022-03-17,69410,68625


In [139]:
contract_df.reset_index(inplace=True)
contract_df.drop('index',inplace=True,axis=1)

In [150]:
#create a list to attach the collected dataframes
notes_data = []

In [163]:
contract_df['notes'] = contract_df.filename.apply(lambda x : get_data(os.path.join(directory,x)))

2022-03-18T18:04:48 - INFO - Processing page-1
2022-03-18T18:04:52 - INFO - Processing page-2
2022-03-18T18:04:56 - INFO - Processing page-3
2022-03-18T18:04:57 - INFO - Processing page-4
2022-03-18T18:05:06 - INFO - Processing page-1
2022-03-18T18:05:10 - INFO - Processing page-2
2022-03-18T18:05:16 - INFO - Processing page-3
2022-03-18T18:05:20 - INFO - Processing page-4
2022-03-18T18:05:22 - INFO - Processing page-5
2022-03-18T18:05:27 - INFO - Processing page-1
2022-03-18T18:05:31 - INFO - Processing page-2
2022-03-18T18:05:35 - INFO - Processing page-3
2022-03-18T18:05:36 - INFO - Processing page-4
2022-03-18T18:05:40 - INFO - Processing page-1
2022-03-18T18:05:43 - INFO - Processing page-2
2022-03-18T18:05:46 - INFO - Processing page-3
2022-03-18T18:05:52 - INFO - Processing page-1
2022-03-18T18:05:56 - INFO - Processing page-2
2022-03-18T18:06:00 - INFO - Processing page-3
2022-03-18T18:06:02 - INFO - Processing page-4
2022-03-18T18:06:07 - INFO - Processing page-1
2022-03-18T18

for index, row in contract_df.iterrows():
    #getting the raw file into the list
    notes_data.append(get_data(os.path.join(directory,row['filename'])))
    print('completed collecting {} data'.format(row['filename']))

In [166]:
contract_df

,filename,dates,Jun_close,Apr_close,notes_data,notes
0,2022-03-07-contract-notes_DM3955.pdf,2022-03-07,70758,70069,order_no script transaction ...,order_no script transaction ...
1,2022-03-08-contract-notes_DM3955.pdf,2022-03-08,72211,71418,order_no script transaction ...,order_no script transaction ...
2,2022-03-09-contract-notes_DM3955.pdf,2022-03-09,70443,69692,order_no script transaction ...,order_no script transaction ...
3,2022-03-10-contract-notes_DM3955.pdf,2022-03-10,71246,70500,order_no script transaction ...,order_no script transaction ...
4,2022-03-11-contract-notes_DM3955.pdf,2022-03-11,71170,70412,order_no script transaction ...,order_no script transaction ...
5,2022-03-14-contract-notes_DM3955.pdf,2022-03-14,69805,68993,order_no script transaction ...,order_no script transaction ...
6,2022-03-15-contract-notes_DM3955.pdf,2022-03-15,69256,68466,order_no script transaction ...,order_no script transaction ...
7,2022-03-16-contract-notes_DM3955.pdf,2022-03-16,68327,67517,order_no script transaction ...,order_no script transaction ...
8,2022-03-17-contract-notes_DM3955.pdf,2022-03-17,69410,68625,order_no script transaction ...,order_no script transaction ...
